In [6]:
import numpy as np
import cv2 as cv

# Load the video
cap = cv.VideoCapture('mot.mp4')

if not cap.isOpened():
    print("Error: Could not open video file.")
    exit()

# Shi-Tomasi corner detection parameters
feature_params = dict(maxCorners=100,
                      qualityLevel=0.3,
                      minDistance=7,
                      blockSize=7)

# Lucas-Kanade optical flow parameters
lk_params = dict(winSize=(15, 15),
                 maxLevel=2,
                 criteria=(cv.TERM_CRITERIA_EPS | cv.TERM_CRITERIA_COUNT, 10, 0.03))

# Create random colors for drawing
color = np.random.randint(0, 255, (100, 3))

# Take the first frame and find corners in it
ret, old_frame = cap.read()
if not ret:
    print("Error: Couldn't read first frame.")
    cap.release()
    exit()

old_gray = cv.cvtColor(old_frame, cv.COLOR_BGR2GRAY)
p0 = cv.goodFeaturesToTrack(old_gray, mask=None, **feature_params)

# Create a mask image for drawing optical flow lines
mask = np.zeros_like(old_frame)

while True:
    ret, frame = cap.read()
    if not ret:
        print('Video ended or failed to grab frame.')
        break

    frame_gray = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)

    # Calculate optical flow
    p1, st, err = cv.calcOpticalFlowPyrLK(old_gray, frame_gray, p0, None, **lk_params)

    # Select good points
    if p1 is not None and st is not None:
        good_new = p1[st == 1]
        good_old = p0[st == 1]

        # Draw the tracks
        for i, (new, old) in enumerate(zip(good_new, good_old)):
            a, b = new.ravel()
            c, d = old.ravel()
            mask = cv.line(mask, (int(a), int(b)), (int(c), int(d)), color[i % len(color)].tolist(), 2)
            frame = cv.circle(frame, (int(a), int(b)), 5, color[i % len(color)].tolist(), -1)

        img = cv.add(frame, mask)

        cv.imshow('Lucas-Kanade Optical Flow', img)

        # Update the previous frame and points
        old_gray = frame_gray.copy()
        p0 = good_new.reshape(-1, 1, 2)

    key = cv.waitKey(30) & 0xFF
    if key == 27:  # ESC to exit
        break

# Cleanup
cap.release()
cv.destroyAllWindows()


In [ ]:
q